In [19]:
# Importamos librerías
from queue import PriorityQueue
import copy
import random


In [20]:
# Definimos el grid original del tablero.
grid: list[list[str]] = [
    ["A", "B", "C", "D", "E"],
    ["F", "G", "H", "I", "J"],
    ["K", "L", "M", "N", "Ñ"],
    ["O", "P", "Q", "R", "S"],
    ["T", "U", "V", "W", "X"],
]

# Definimos la posición de cada coordenada
locations: dict[str, tuple[int, int]] = {}

# Iterando sobre cada fila y columna del para agregar la letra y su ubicación al diccionario
for i, row in enumerate(grid):
    for j, letter in enumerate(row):
        locations[letter] = (i, j)

print(locations)


{'A': (0, 0), 'B': (0, 1), 'C': (0, 2), 'D': (0, 3), 'E': (0, 4), 'F': (1, 0), 'G': (1, 1), 'H': (1, 2), 'I': (1, 3), 'J': (1, 4), 'K': (2, 0), 'L': (2, 1), 'M': (2, 2), 'N': (2, 3), 'Ñ': (2, 4), 'O': (3, 0), 'P': (3, 1), 'Q': (3, 2), 'R': (3, 3), 'S': (3, 4), 'T': (4, 0), 'U': (4, 1), 'V': (4, 2), 'W': (4, 3), 'X': (4, 4)}


In [21]:
# Definimos la función heurística "h(s)" que obtiene la distancia de Manhattan
def manhattan_distance(start: str, goal: str, locs: dict[str, tuple[int, int]] ):
    x1, y1 = locs[start]
    x2, y2 = locs[goal]
    # Se calcula la distancia absoluta entre las casillas
    return abs(x1 - x2) + abs(y1 - y2)

In [22]:
# Obtenemos los vecinos para cada nodo
def get_neighbors(node: str, grd: list[list[str]], locs: dict[str, tuple[int, int]] ):
    x, y = locs[node]
    neighbors = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Arriba, Abajo, Izq, Der
    # Se revisa cada coordenada, a partir de los offsets de 'directions'
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        # Si la coordenada a revisar no está fuera del borde y no está vacía, entonces la añadimos a los vecinos
        if 0 <= nx < len(grd) and 0 <= ny < len(grd[0]) and grd[nx][ny] != ' ':
            neighbors.append(grd[nx][ny])
    return neighbors

In [23]:
# Greedy search
def greedy_search(start: str, goal: str, grd: list[list[str]], locs: dict[str, tuple[int, int]]):
    # Creamos la fila de prioridad
    frontier = PriorityQueue()
    frontier.put((0, start))
    came_from = {start: None}

    # Mientras la fila de prioridad NO está vacía
    while not frontier.empty():
        # Obtenemos la casilla con el menor costo heurístico
        current = frontier.get()[1]

        print("---------------------------")
        print("Nodo actual:", current)

        # Si la casilla actual es la meta, se acaba el algoritmo
        if current == goal:
            break

        # Checamos TODOS los vecinos en el nodo actual
        for next_node in get_neighbors(current, grd, locs):
            if next_node not in came_from:
                priority = manhattan_distance(next_node, goal, locs)
                frontier.put((priority, next_node))
                came_from[next_node] = current

    # Reconstruimos el path desde el nodo actual hasta la meta
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path

In [24]:
# Definición principal del programa
def main():
    # Ingreso de casillas de inicio y final
    inicio = input("Ingresar la casilla de inicio: ")
    final = input("Ingresar la casilla final: ")

    # Si son la misma casilla, terminamos, pues no vamos a buscar un camino.
    if final == inicio:
        print("El inicio y el final son iguales, abortando.")
        return

    # Creamos copias nuevas de grid y locations, para modificarlos librementes y dejar los tableros originales intactos
    new_grid = copy.deepcopy(grid)
    new_locs = copy.deepcopy(locations)

    # Tomamos una muestra al azar de 6 elementos únicos (que no sean ni la casilla de inicio ni la del final) que servirán como obstáculos.
    a_eliminar = random.sample([k for k in new_locs.keys() if k != inicio and k != final], k=6)

    print("Grid original:", new_grid)
    print("Las casillas obstáculo son:", a_eliminar)

    # Eliminamos del new_grid las 6 casillas elegidas al azar
    for i, row in enumerate(new_grid):
        for j, el in enumerate(row):
            if el in a_eliminar:
                new_grid[i][j] = " "
    
    print("Grid modificado:", new_grid )

    # Se ejecuta la búsqueda y se obtiene el resultado
    path = greedy_search(inicio, final, new_grid, new_locs)
    print("Camino final: ", path)

# Ejecución del programa
main()

Grid original: [['A', 'B', 'C', 'D', 'E'], ['F', 'G', 'H', 'I', 'J'], ['K', 'L', 'M', 'N', 'Ñ'], ['O', 'P', 'Q', 'R', 'S'], ['T', 'U', 'V', 'W', 'X']]
Las casillas obstáculo son: ['T', 'M', 'E', 'D', 'K', 'R']
Grid modificado: [['A', 'B', 'C', ' ', ' '], ['F', 'G', 'H', 'I', 'J'], [' ', 'L', ' ', 'N', 'Ñ'], ['O', 'P', 'Q', ' ', 'S'], [' ', 'U', 'V', 'W', 'X']]
---------------------------
Nodo actual: A
---------------------------
Nodo actual: B
---------------------------
Nodo actual: C
---------------------------
Nodo actual: H
---------------------------
Nodo actual: G
---------------------------
Nodo actual: L
---------------------------
Nodo actual: P
---------------------------
Nodo actual: Q
Camino final:  ['A', 'B', 'G', 'L', 'P', 'Q']
